In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
import torch
import torch.nn as nn 

from model.SpeechLP import SLP

from utils.Config import Config
from utils.MLS import MLSDataset
from utils.Trainer import Trainer

from torch.utils.data import DataLoader, random_split

In [63]:
Config.display()

Audio Settings:
  Sample Rate: 24000 Hz
  Min Audio Duration: 10 seconds
  Max Audio Duration: 20 seconds

Text Settings:
  Max Token : 128 tokens

Model Settings:
  Model Name: DiTTO-TTS
  Embedding Dim: 1472
  Num Layers: 1
  Attention Heads: 1

Training Settings:
  Batch Size: 8
  Learning Rate: 0.0001
  Betas: [0.9, 0.999]
  Epochs: 10
  Nb samples: 10000
  Device: cuda

Data Settings:
  Train path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/train
  Test path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/test
  Dev path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/dev


In [64]:
train_set = MLSDataset(
    data_dir=Config.TRAIN_PATH,
    max_text_token_length=Config.MAX_TOKEN_LENGTH,
    sampling_rate=Config.SAMPLE_RATE,
)

val_set = MLSDataset(
    data_dir=Config.DEV_PATH,
    max_text_token_length=Config.MAX_TOKEN_LENGTH,
    sampling_rate=Config.SAMPLE_RATE,
)


train_loader = DataLoader(train_set, batch_size=Config.BATCH_SIZE, shuffle=True, collate_fn=MLSDataset.collate_fn)
val_loader = DataLoader(val_set, batch_size=Config.BATCH_SIZE, shuffle=True, collate_fn=MLSDataset.collate_fn)

In [66]:
from tqdm import tqdm

for i, batch in enumerate(tqdm(val_loader)):
    try:
        # Process the batch as normal
        text = {key: val.to(Config.DEVICE) for key, val in batch["text"].items()}
    except UnicodeDecodeError as e:
        print(f"[ERROR] UnicodeDecodeError in batch {i}: {e}")
        break



  0%|          | 0/302 [00:00<?, ?it/s]

  0%|          | 0/302 [00:00<?, ?it/s]


[ERROR] UnicodeDecodeError at index 2007: 'utf-8' codec can't decode byte 0xff in position 10: invalid start byte
Audio path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/dev/audio/7591/6742/7591_6742_000068.opus
Transcript: neuf la pluie la pluie fine a mouillé toutes choses très doucement et en silence il pleut encore un peu je vais sortir sous les arbres pieds nus pour ne pas tacher mes chaussures


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 10: invalid start byte

In [37]:
model = SLP(Config.MAX_AUDIO_DURATION, Config.NHEAD ,Config.NUM_LAYERS).to(Config.DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW

trainer = Trainer()
trainer.set_model(model, name="SLP")\
    .set_criterion(criterion)\
    .set_optimizer(optimizer)\
    .fit(
        train_data=train_loader, validation_data=val_loader, 
        epochs=Config.EPOCHS, learning_rate=Config.LEARNING_RATE, checkpoint_interval=1        
    )

[INFO] Model's device is : cuda


  0%|          | 0/10 [00:00<?, ?it/s]


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe8 in position 9: invalid continuation byte